In [50]:
from mintalib.samples import sample_prices


%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [51]:
prices = sample_prices()
prices.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11056 entries, 1980-12-12 to 2024-10-21
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    11056 non-null  float64
 1   high    11056 non-null  float64
 2   low     11056 non-null  float64
 3   close   11056 non-null  float64
 4   volume  11056 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 518.2 KB


In [52]:
# Definitions to remove missing definitions warnings

def np_sma(xs, period: int):
    pass

def calc_sma(series, period: int, wrap: bool=False):
    pass

def SMA(series, period: int):
    pass


In [53]:
%%cython -c=-Wno-unreachable-code

import sys
import numpy as np

from libc.math cimport isnan

cdef double NAN = float('nan')


def wrap_result(result, source):
    """ wrap result to match source data form (pandas, polars) """

    pname = getattr(source, '__module__', '').partition('.')[0]

    if isinstance(result, tuple) and hasattr(result, '_asdict'):
        result = result._asdict()

    if pname == 'pandas':
        pandas = sys.modules['pandas']
        index = getattr(source, 'index', None)

        if isinstance(result, dict):
            return pandas.DataFrame(result, index=index)

        if isinstance(result, np.ndarray):
            return pandas.Series(result, index=index)

    if pname == 'polars':
        polars = sys.modules['polars']

        if isinstance(result, dict):
            return polars.DataFrame(result)

        if isinstance(result, np.ndarray):
            return polars.Series(result)

    return result



def calc_sma(series, long period, bint wrap = False):
    """ simple moving average """

    cdef double[:] xs = np.asarray(series, float)
    cdef long size = xs.size

    cdef object result = np.full(size, np.nan)
    cdef double[:] output = result

    cdef double v = NAN, rsum = 0
    cdef long i = 0, j = 0, count = 0

    for i in range(size):
        v = xs[i]

        if not isnan(v):
            rsum += v
            count += 1

        while count > period and j < i:
            v, j = xs[j], j+1
            if not isnan(v):
                rsum -= v
                count -= 1

        if count >= period:
            output[i] = rsum / count

    if wrap:
        result = wrap_result(result, series)

    return result

print("done!")

In [54]:
calc_sma(prices.close, 5)

array([         nan,          nan,          nan, ..., 231.32600098,
       232.81600037, 233.8519989 ])

In [55]:
%%cython -c=-Wno-unreachable-code

# cython: language_level=3, binding=True

import sys

import numpy as np

from functools import wraps

from libc.math cimport isnan

# from mintalib.core import wrap_result

cdef double NAN = float('nan')


def dataframe_like(data):
    """ check if data is dataframe like """

    if isinstance(data, dict):
        return True

    if isinstance(data, np.ndarray):
        return data.dtype.names is not None

    if hasattr(data, 'columns'):
        return True

    return  False


def get_series(data, item: str = None, *, default_item: str = 'close'):
    """ get series from either series/prices data """

    if dataframe_like(data):
        if item is None:
            item = default_item
        return data[item]

    if item is not None:
        tname = type(data).__name__
        raise ValueError(f"Cannot get series from {tname} data")

    return data


def wrap_result(result, source):
    """ wrap result to match source data form (pandas, polars) """

    pname = getattr(source, '__module__', '').partition('.')[0]

    if isinstance(result, tuple) and hasattr(result, '_asdict'):
        result = result._asdict()

    if pname == 'pandas':
        pandas = sys.modules['pandas']
        index = getattr(source, 'index', None)

        if isinstance(result, dict):
            return pandas.DataFrame(result, index=index)

        if isinstance(result, np.ndarray):
            return pandas.Series(result, index=index)

    if pname == 'polars':
        polars = sys.modules['polars']

        if isinstance(result, dict):
            return polars.DataFrame(result)

        if isinstance(result, np.ndarray):
            return polars.Series(result)

    return result


cpdef np_sma(double[:] xs, long period):
    """ simple moving average """

    cdef long size = xs.size

    cdef object result = np.full(size, np.nan)
    cdef double[:] output = result

    cdef double v = NAN, rsum = 0
    cdef long i = 0, j = 0, count = 0

    for i in range(size):
        v = xs[i]

        if not isnan(v):
            rsum += v
            count += 1

        while count > period and j < i:
            v, j = xs[j], j+1
            if not isnan(v):
                rsum -= v
                count -= 1

        if count >= period:
            output[i] = rsum / count

    # also return output.base
    return result 


def calc_sma(series, int period, bint wrap = False):
    """ simple moving average """

    cdef double[:] xs = np.asarray(series, float)
    cdef object result = np_sma(xs, period)

    if wrap:
        result = wrap_result(result, series)

    return result



def make_function(func):
    def decorator(target):
        @wraps(target)
        def wrapper(prices, *args, **kwargs):
            series = get_series(prices)
            result = func(series, *args, **kwargs)
            return wrap_result(result, prices)
        return wrapper
    return decorator


@make_function(calc_sma)
def SMA(series, period: int = 20):
    pass


print("done!")

In [56]:
res = np_sma(prices.close.values, 5)
print(type(res))
res

<class 'numpy.ndarray'>


array([         nan,          nan,          nan, ..., 231.32600098,
       232.81600037, 233.8519989 ])

In [57]:
res = calc_sma(prices.close, 3)
print(type(res))
res

<class 'numpy.ndarray'>


array([           nan,            nan, 9.32074760e-02, ...,
       2.32593333e+02, 2.32976664e+02, 2.34543330e+02])

In [58]:
res = calc_sma(prices.close, 3, wrap=True)
print(type(res))
res

<class 'pandas.core.series.Series'>


date
1980-12-12           NaN
1980-12-15           NaN
1980-12-16      0.093207
1980-12-17      0.089909
1980-12-18      0.089192
                 ...    
2024-10-15    230.900004
2024-10-16    232.310003
2024-10-17    232.593333
2024-10-18    232.976664
2024-10-21    234.543330
Length: 11056, dtype: float64

In [59]:
help(calc_sma)

Help on cython_function_or_method in module _cython_magic_b2d1634a343669d0fb52d5a849588be0da328ec5:

calc_sma(series, period, wrap=False)
    simple moving average



In [60]:
SMA


<cyfunction SMA at 0x128d756c0>

In [61]:
SMA(prices, 20)


date
1980-12-12           NaN
1980-12-15           NaN
1980-12-16           NaN
1980-12-17           NaN
1980-12-18           NaN
                 ...    
2024-10-15    227.524000
2024-10-16    228.078500
2024-10-17    228.242500
2024-10-18    228.582500
2024-10-21    229.082999
Length: 11056, dtype: float64